### Imports ###

In [2]:
import os,sys
import http.client
import json
from datetime import datetime
import pandas as pd

### Statics ###

Folders

In [3]:
resources=os.path.join(os.path.abspath('..'),'resources','csv')

API

In [4]:
# Football API
FOOTBALL_API_KEY = 'cbc491cd36f8a38ff058fe85f3265b8b'
URL = 'https://v3.football.api-sports.io'
api_football = http.client.HTTPSConnection("v3.football.api-sports.io")

# FUT API
FUT_API_KEY = 'f64149b1-ecbe-46c1-9b7f-60ae79dac51e'

Dates

In [5]:
today= datetime.strftime(datetime.now(),'%Y-%m-%d')

Football

In [7]:
top_leagues = [2,3,39,61,78,88,135,140]

#### Base data ####

Leagues

In [80]:
# Set target file for leagues
target_file = os.path.join(resources,'leagues.csv')

# If the leagues file is missing, then get it again from the API
if os.path.isfile(target_file)==False:
    api_football = http.client.HTTPSConnection("v3.football.api-sports.io")
    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': FOOTBALL_API_KEY
        }
    params = f""

    api_football.request("GET", f"/leagues?{params}", headers=headers)

    res = api_football.getresponse()
    data = res.read()
    data_json = json.loads(data)
    
    all_rows=[]
    for i in j['response']:
        row={}
        row['id'] = i['league']['id']
        row['name'] = i['league']['name']
        row['country'] = i['country']['name']
        row['first_year'] = i['seasons'][0]['year']
        row['current']= ''
        for ii in i['seasons']:
            if ii['current']==True:
                row['current'] = ii['year']
        
        all_rows.append(row)

    leagues = pd.DataFrame(all_rows)
    leagues.to_csv(target_file,index=False)
else:
    # Otherwise read the file
    leagues=pd.read_csv(target_file)

Teams

In [81]:
# Set target file for leagues
target_file = os.path.join(resources,'teams.csv')

# Target connection
api_football = http.client.HTTPSConnection("v3.football.api-sports.io")

# If the leagues file is missing, then get it again from the API
if os.path.isfile(target_file)==False:
    all_rows = []
    for i,row in leagues.iterrows():
        league_id = row['id']
        league_season = row['current']
        league_name = row['name']
        if league_id in top_leagues:
            headers = {
                'x-rapidapi-host': "v3.football.api-sports.io",
                'x-rapidapi-key': FOOTBALL_API_KEY
                }
            params = f"league={league_id}&&season={league_season}"

            api_football.request("GET", f"/teams?{params}", headers=headers)

            res = api_football.getresponse()
            data = res.read()
            data_json = json.loads(data)
            for i in data_json['response']:
                row = {}
                row['id'] = i['team']['id']
                row['name'] = i['team']['name']
                row['code'] = i['team']['code']
                row['league_id'] = league_id
                row['league_name'] = league_name
                row['league_season'] = league_season
                row['venue_id'] = i['venue']['id']
                row['venue_name'] = i['venue']['name']
                row['venue_capacity'] = i['venue']['capacity']
                all_rows.append(row)

    teams = pd.DataFrame(all_rows)
    teams.to_csv(target_file,index=False)
else:
    # Otherwise read the file
    teams=pd.read_csv(target_file)

Fixtures

In [82]:
api_football = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': FOOTBALL_API_KEY
    }
params = f"date={today}"

api_football.request("GET", f"/fixtures?{params}", headers=headers)

res = api_football.getresponse()
data = res.read()
data_json = json.loads(data)

all_rows = []
for i in data_json['response']:
    row = {}
    row['status'] = i['fixture']['status']['long']
    row['date'] = i['fixture']['date']
    row['timestamp'] = i['fixture']['timestamp']
    row['id'] = i['fixture']['id']
    row['league_id'] = i['league']['id']
    row['league_name'] = i['league']['name']
    row['home_id'] = i['teams']['home']['id']
    row['home_name'] = i['teams']['home']['name']
    row['away_id'] = i['teams']['away']['id']
    row['away_name'] = i['teams']['away']['name']
    all_rows.append(row)

fixtures = pd.DataFrame(all_rows)
fixtures['date'] =  pd.to_datetime(fixtures['date'])
fixtures = fixtures[fixtures['league_id'].isin(top_leagues)]

Target game

In [83]:
target_game = 871344

Getting selected fixture

In [84]:
home_team = fixtures[fixtures['id']==target_game]['home_id'].min()
away_team = fixtures[fixtures['id']==target_game]['away_id'].min()
league = fixtures[fixtures['id']==target_game]['league_id'].min()
season = teams[teams['league_id']==league]['league_season'].min()

this_match = [
    {'team':'home','id':home_team},
    {'team':'away','id':away_team},
                ]

In [85]:
for i in this_match:
    print(i['team'])

home
away


Get the two teams stats

In [86]:
for t in this_match:
    id = t['id']
    team = t['team']
    api_football = http.client.HTTPSConnection("v3.football.api-sports.io")

    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': FOOTBALL_API_KEY
        }
    params = f"season={season}&team={id}&league={league}"

    api_football.request("GET", f"/teams/statistics?{params}", headers=headers)

    res = api_football.getresponse()
    data = res.read()
    data_json = json.loads(data)

    all_rows = []
    i= data_json['response']
    row ={}
    row['id']=i['team']['id']
    row['name']=i['team']['name']
    row['form']=i['form']
    row['played']=i['fixtures']['played']['total']
    row['played_home']=i['fixtures']['played']['home']
    row['played_away']=i['fixtures']['played']['away']
    row['wins']=i['fixtures']['wins']['total']
    row['wins_home']=i['fixtures']['wins']['home']
    row['wins_away']=i['fixtures']['wins']['away']
    row['draws']=i['fixtures']['draws']['total']
    row['draws_home']=i['fixtures']['draws']['home']
    row['draws_away']=i['fixtures']['draws']['away']
    row['loses']=i['fixtures']['loses']['total']
    row['loses_home']=i['fixtures']['loses']['home']
    row['loses_away']=i['fixtures']['loses']['away']
    row['goals_for']=i['goals']['for']['total']['total']
    row['goals_for_home']=i['goals']['for']['total']['home']
    row['goals_for_away']=i['goals']['for']['total']['away']
    row['goals_for_avg']=i['goals']['for']['average']['total']
    row['goals_for_avg_home']=i['goals']['for']['average']['home']
    row['goals_for_avg_away']=i['goals']['for']['average']['away']
    row['goals_for_min_15']=i['goals']['for']['minute']['0-15']['percentage']
    row['goals_for_min_30']=i['goals']['for']['minute']['16-30']['percentage']
    row['goals_for_min_45']=i['goals']['for']['minute']['31-45']['percentage']
    row['goals_for_min_60']=i['goals']['for']['minute']['46-60']['percentage']
    row['goals_for_min_75']=i['goals']['for']['minute']['61-75']['percentage']
    row['goals_for_min_90']=i['goals']['for']['minute']['76-90']['percentage']
    row['goals_against']=i['goals']['against']['total']['total']
    row['goals_against_home']=i['goals']['against']['total']['home']
    row['goals_against_away']=i['goals']['against']['total']['away']
    row['goals_against_avg']=i['goals']['against']['average']['total']
    row['goals_against_avg_home']=i['goals']['against']['average']['home']
    row['goals_against_avg_away']=i['goals']['against']['average']['away']
    row['goals_against_min_15']=i['goals']['against']['minute']['0-15']['percentage']
    row['goals_against_min_30']=i['goals']['against']['minute']['16-30']['percentage']
    row['goals_against_min_45']=i['goals']['against']['minute']['31-45']['percentage']
    row['goals_against_min_60']=i['goals']['against']['minute']['46-60']['percentage']
    row['goals_against_min_75']=i['goals']['against']['minute']['61-75']['percentage']
    row['goals_against_min_90']=i['goals']['against']['minute']['76-90']['percentage']
    row['clean_sheet']=i['clean_sheet']['total']
    row['clean_sheet_home']=i['clean_sheet']['home']
    row['clean_sheet_away']=i['clean_sheet']['away']
    row['failed_to_score']=i['failed_to_score']['total']
    row['failed_to_score_home']=i['failed_to_score']['home']
    row['failed_to_score_away']=i['failed_to_score']['away']

    all_rows.append(row)

    if team=='home':
        home = pd.DataFrame(all_rows).melt()
    else:
        away = pd.DataFrame(all_rows).melt()

home['team'] = 'home'
away['team'] = 'away'
match_results = pd.concat([home,away])
match_results = match_results.pivot_table(index='variable',columns='team',values='value',aggfunc='sum').reset_index()

TypeError: list indices must be integers or slices, not str

Last few games

In [ ]:
for i in [1,3,5,10,20]:
    row = pd.DataFrame({
        'variable':f'last {i} games - wins',
        'home': match_results.query('variable=="form"')['home'].min()[:i].count('W'),
        'away': match_results.query('variable=="form"')['away'].min()[:i].count('W')
    },index=[len(match_results)])
    match_results = pd.concat([match_results,row])

### FUT ###

Players

In [100]:
# Set target file for leagues
target_file = os.path.join(resources,'players.csv')

# If the leagues file is missing, then get it again from the API
if os.path.isfile(target_file)==False:
    api_fut = http.client.HTTPSConnection("futdb.app")
    n=1
    total_pages = 2
    headers = {
        'accept': "application/json",
        'X-AUTH-TOKEN': FUT_API_KEY
        }
    all_rows = []
    while n<=total_pages:
        params = f"page={n}"

        api_fut.request("GET", f"/api/players?{params}", headers=headers)

        res = api_fut.getresponse()
        data = res.read()
        data_json = json.loads(data)
        if n==1:
            total_pages = data_json['pagination']['pageTotal']

        for i in data_json['items']:
            row = {}
            row['id'] = i['id']
            row['firstName'] = i['firstName']
            row['lastName'] = i['lastName']
            row['name'] = i['name']
            row['birthdate'] = i['birthDate']
            row['club'] = i['club']
            row['league'] = i['league']
            row['nation'] = i['nation']
            row['height'] = i['height']
            row['weight'] = i['weight']
            row['color'] = i['color']
            row['rarity'] = i['rarity']
            row['position'] = i['position']
            row['skillMoves'] = i['skillMoves']
            row['weakFoot'] = i['weakFoot']
            row['totalStats'] = i['totalStats']
            row['rating'] = i['rating']
            row['ratingAverage'] = i['ratingAverage']
            row['pace'] = i['pace']
            row['shooting'] = i['shooting']
            row['passing'] = i['passing']
            row['dribbling'] = i['dribbling']
            row['defending'] = i['defending']
            row['physicality'] = i['physicality']
            all_rows.append(row)

        n+=1
    players = pd.DataFrame(all_rows)
    players.to_csv(target_file,index=False)
else:
    players = pd.read_csv(target_file)

Clubs

In [101]:
# Set target file for leagues
target_file = os.path.join(resources,'fut_clubs.csv')

# If the leagues file is missing, then get it again from the API
if os.path.isfile(target_file)==False:
    api_fut = http.client.HTTPSConnection("futdb.app")
    n=1
    total_pages = 2
    headers = {
        'accept': "application/json",
        'X-AUTH-TOKEN': FUT_API_KEY
        }
    all_rows = []
    while n<=total_pages:
        params = f"page={n}"

        api_fut.request("GET", f"/api/clubs?{params}", headers=headers)

        res = api_fut.getresponse()
        data = res.read()
        data_json = json.loads(data)
        if n==1:
            total_pages = data_json['pagination']['pageTotal']

        for i in data_json['items']:
            row = {}
            row['id'] = i['id']
            row['name'] = i['name']
            row['league'] = i['league']
            all_rows.append(row)

        n+=1
    fut_clubs = pd.DataFrame(all_rows)
    fut_clubs.to_csv(target_file,index=False)
else:
    fut_clubs = pd.read_csv(target_file)

Leagues

In [102]:
# Set target file for leagues
target_file = os.path.join(resources,'fut_leagues.csv')

# If the leagues file is missing, then get it again from the API
if os.path.isfile(target_file)==False:
    api_fut = http.client.HTTPSConnection("futdb.app")
    n=1
    total_pages = 2
    headers = {
        'accept': "application/json",
        'X-AUTH-TOKEN': FUT_API_KEY
        }
    all_rows = []
    while n<=total_pages:
        params = f"page={n}"

        api_fut.request("GET", f"/api/leagues?{params}", headers=headers)

        res = api_fut.getresponse()
        data = res.read()
        data_json = json.loads(data)
        if n==1:
            total_pages = data_json['pagination']['pageTotal']

        for i in data_json['items']:
            row = {}
            row['id'] = i['id']
            row['name'] = i['name']
            all_rows.append(row)

        n+=1
    fut_leagues = pd.DataFrame(all_rows)
    fut_leagues.to_csv(target_file,index=False)
else:
    fut_leagues = pd.read_csv(target_file)

Nations

In [103]:
# Set target file for leagues
target_file = os.path.join(resources,'fut_nations.csv')

# If the leagues file is missing, then get it again from the API
if os.path.isfile(target_file)==False:
    api_fut = http.client.HTTPSConnection("futdb.app")
    n=1
    total_pages = 2
    headers = {
        'accept': "application/json",
        'X-AUTH-TOKEN': FUT_API_KEY
        }
    all_rows = []
    while n<=total_pages:
        params = f"page={n}"

        api_fut.request("GET", f"/api/nations?{params}", headers=headers)

        res = api_fut.getresponse()
        data = res.read()
        data_json = json.loads(data)
        if n==1:
            total_pages = data_json['pagination']['pageTotal']

        for i in data_json['items']:
            row = {}
            row['id'] = i['id']
            row['name'] = i['name']
            all_rows.append(row)

        n+=1
    fut_nations = pd.DataFrame(all_rows)
    fut_nations.to_csv(target_file,index=False)
else:
    fut_nations = pd.read_csv(target_file)

In [106]:
fut_clubs = fut_clubs.rename({'id':'club_id','name':'club_name','league':'league_id'},axis=1)
fut_nations = fut_nations.rename({'id':'nation_id','name':'nation_name'},axis=1)
fut_leagues = fut_leagues.rename({'id':'league_id','name':'league_name'},axis=1)
players = players.merge(fut_clubs,how='left',left_on='club',right_on='club_id')
players = players.merge(fut_leagues,how='left',left_on='league',right_on='league_id')
players = players.merge(fut_nations,how='left',left_on='nation',right_on='nation_id')
players.drop(['club','league','nation','club_id','league_id_x','league_id_y','nation_id'],axis=1,inplace=True)
players = players[players['league_name']!='Icons']

In [116]:
players.query("club_name=='Union Berlin'").sort_values('rating',ascending=False).drop_duplicates('name')['rating'].mean()

72.125

In [128]:
players[players['club_name'].str.contains('Dortmund')].sort_values('rating',ascending=False).drop_duplicates('name')['rating'].mean()

70.69642857142857

In [129]:
players

,id,firstName,lastName,name,height,weight,birthdate,position,skillMoves,weakFoot,...,ratingAverage,pace,shooting,passing,dribbling,defending,physicality,club_name,league_name,nation_name
1,17227,NaN,NaN,Lionel Messi,169,67,1987-06-24,RW,4,4,...,0,0,0,0,0,0,0,Paris SG,Ligue 1,Argentina
2,17224,NaN,NaN,Kevin De Bruyne,181,70,1991-06-28,CM,4,5,...,0,0,0,0,0,0,0,Manchester City,Premier League,Belgium
3,17225,NaN,NaN,Kylian Mbappé,182,73,1998-12-20,ST,5,4,...,0,0,0,0,0,0,0,Paris SG,Ligue 1,France
4,17226,NaN,NaN,Karim Benzema,185,81,1987-12-19,CF,4,4,...,0,0,0,0,0,0,0,Real Madrid,LaLiga Santander,France
7,17223,NaN,NaN,Luka Modric,172,66,1985-09-09,CM,4,4,...,0,0,0,0,0,0,0,Real Madrid,LaLiga Santander,Croatia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17556,14790,Ziye,Zhao,Ziye Zhao,180,0,2003-10-01,RW,2,3,...,44,63,48,40,47,24,44,Hebei CFFC,Chinese FA Super L. (CHN 1),China PR
17557,15077,Antonio,D'Silva,Antonio D'Silva,182,0,2000-04-04,GK,1,3,...,42,19,13,17,28,11,32,Odisha FC,Indian Super League (IND 1),India
17558,13807,Junjie,Wu,Junjie Wu,188,0,2003-06-02,LB,2,3,...,41,55,25,29,34,48,57,Guangzhou R&F,Chinese FA Super L. (CHN 1),China PR
17559,15099,Devansh,Dabas,Devansh Dabas,182,0,2001-09-10,GK,1,2,...,43,28,13,19,26,10,39,Chennaiyin FC,Indian Super League (IND 1),India
